In [1]:
import pandas as pd
import numpy as np
import requests
import regex as re
from bs4 import BeautifulSoup

In [2]:
links = pd.read_csv('link.csv',header=None)
links

,0
0,https://www.coindesk.com/markets/2021/08/04/bi...
1,https://www.coindesk.com/markets/2022/01/31/fi...
2,https://www.coindesk.com/markets/2022/02/01/ha...
3,https://www.newsbtc.com/news/bitcoin/market-se...
4,https://www.newsbtc.com/analysis/btc/ta-bitcoi...
5,https://bitcoinmagazine.com/business/at-home-b...
6,https://bitcoinmagazine.com/culture/why-should...
7,https://cryptoslate.com/whats-behind-bitcoins-...
8,https://cryptoslate.com/fans-can-now-use-ether...
9,https://bitcoinist.com/why-deustche-bank-says-...


In [3]:
links = links.drop_duplicates()
links = links[links[0].str.contains('coindesk')]
links

,0
0,https://www.coindesk.com/markets/2021/08/04/bi...
1,https://www.coindesk.com/markets/2022/01/31/fi...
2,https://www.coindesk.com/markets/2022/02/01/ha...


In [4]:
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "Accept-Language": "en-US,en;q=0.5", "Accept-Encoding": "gzip, deflate", "DNT": "1", "Connection": "close", "Upgrade-Insecure-Requests": "1"}

In [5]:
links[0][0]

'https://www.coindesk.com/markets/2021/08/04/bitcoin-stuck-below-40k-eyes-short-term-oversold-bounce/'

In [6]:
# Create lists that contains article and date scrapped from each link
articles = []
dates = []
scrapped_links = []

# Initialize a count variable to keep track of how many news articles have been scrapped so far
count = 0

# for link in links[0].iloc[2] :
for link in links[0] :
    URL = link
    # Request access to the URL
    page = requests.get(URL, headers=headers)

    # Parse the html using BeautifulSoup for scrapping
    soup = BeautifulSoup(page.content, "html.parser")
    #     result = soup.find_all('p')

    # Find the time in the the `time` class in the html
    date = soup.find_all('span', {'class':'typography__StyledTypography-owin6q-0 dHSCiD'})
    date = re.sub(',|at|UTC', '', date[279].text)
    date = re.sub(' a.m.', ' AM', date)
    date = re.sub(' p.m.', ' PM', date)
    date = re.sub('  ', ' ', date)

    # Request access to the URL
    page = requests.get(URL, headers=headers)

    # Parse the html using BeautifulSoup for scrapping
    soup = BeautifulSoup(page.content, "html.parser")
    result = soup.find_all('p')

    text = []
    for r in result :
        r = str(r)
        if (r.startswith('<p>')) and (r.endswith('</p>')) :
            r = re.sub('<p>|</p>|<b>|</b>|<a |</a>|</p>|<i>|</i>|<br>|</br>|<br/>|<|>','',r)
            text.append(r)
            
    if text :
        dates.append(date)
        articles.append(''.join(text))

In [9]:
dates

['Aug 4 2021 11:31 AM ', 'Jan 31 2022 11:30 PM ', 'Feb 1 2022 7:55 AM ']

In [10]:
len(dates), len(articles)

(3, 3)

In [11]:
for a in articles :
    print(a, end='\n\n')

Bitcoin (href="https://www.coindesk.com/price/bitcoin"BTC) buyers continue to struggle below $40,000 resistance as the short-squeeze rally fades. The cryptocurrency was trading at around $38,000 at press time and is down 4% over the past week. Short-term oversold signals suggest the pullback could stabilize above $36,000 support.Bitcoin (href="https://www.coindesk.com/price/bitcoin"BTC) buyers continue to struggle below $40,000 resistance as the short-squeeze rally fades. The cryptocurrency was trading at around $38,000 at press time and is down 4% over the past week. Short-term oversold signals suggest the pullback could stabilize above $36,000 support.

Good morning. Here’s what’s happening:Market moves: Bitcoin ended a bad January on a positive note; DeFi trading volume grew steadily.Technician's take: BTC's January sell-off could attract short-term buyers.Catch the latest episodes of href="https://www.coindesk.com/tv/" target="_blank"CoinDesk TV for insightful interviews with crypt

In [21]:
links['DateTime'] = dates
links

,0,DateTime
0,https://www.coindesk.com/markets/2021/08/04/bi...,Aug 4 2021 11:31 AM
1,https://www.coindesk.com/markets/2022/01/31/fi...,Jan 31 2022 11:30 PM
2,https://www.coindesk.com/markets/2022/02/01/ha...,Feb 1 2022 7:55 AM


In [22]:
links['DateTime'] = pd.to_datetime(links['DateTime'])
links

,0,DateTime
0,https://www.coindesk.com/markets/2021/08/04/bi...,2021-08-04 11:31:00
1,https://www.coindesk.com/markets/2022/01/31/fi...,2022-01-31 23:30:00
2,https://www.coindesk.com/markets/2022/02/01/ha...,2022-02-01 07:55:00


In [23]:
links['Date'] = links['DateTime'].dt.date
links['Time'] = links['DateTime'].dt.round('H').dt.time
links

,0,DateTime,Date,Time
0,https://www.coindesk.com/markets/2021/08/04/bi...,2021-08-04 11:31:00,2021-08-04,12:00:00
1,https://www.coindesk.com/markets/2022/01/31/fi...,2022-01-31 23:30:00,2022-01-31,00:00:00
2,https://www.coindesk.com/markets/2022/02/01/ha...,2022-02-01 07:55:00,2022-02-01,08:00:00
